In [4]:
import numpy as np
import pandas as pd
import os
times = [0,8,13,20]
response=[]
satisfy=[]
citys=os.listdir('data')
for city in citys:
    files=os.listdir('data/'+city)
    for file in files:
        f=pd.read_csv('data/'+city+'/'+file)
        if file.startswith('response'):
            response.append(sum(list(f.value))/len(list(f.value)))
        elif file.startswith('satisfy'):
            satisfy.append(sum(list(f.value))/len(list(f.value)))            
    break
#读取北京市的数据

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb8 in position 0: invalid start byte

In [ ]:
response

In [ ]:
satisfy

In [66]:
data=[]
for i in range(len(response)):
    data.append([response[i],satisfy[i]])

In [142]:
#训练网络
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

class MyDataset(Dataset):
    def __init__(self,dataset):
        self.full_batch=dataset
        
    def __len__(self):
        return len(self.full_batch)
    
    def __getitem__(self,index):
        response=self.full_batch[index][0]
        satisfy=self.full_batch[index][1]
        return  response,satisfy

class Model(nn.Module):
    def __init__(self, num):
        super(Model, self).__init__()
        self.l1=nn.Linear(1,num)
        self.l2=nn.Linear(num,1)
    def forward(self,x):
        x=self.l1(x)
        x=F.tanh(x)
        x=self.l2(x)
        x=F.sigmoid(x)
        return x

dataset=MyDataset(data)
model=Model(5)
dataloader=DataLoader(dataset,shuffle=True,batch_size=7)
optimizer=optim.Adam(model.parameters(),lr=0.001)

loss_seq=[]
for i in range(10000):
    for response,satisfy in dataloader:
        optimizer.zero_grad()
        response=response.resize(7,1).float()
        satisfy=satisfy.float()/10
        y=model(response)
        y=y.squeeze()
        loss=torch.mean((y-satisfy).pow(2))
        loss.backward()
        optimizer.step()
        loss_seq.append(float(loss))

In [1]:
#拟合结果
x=np.linspace(30,45,50)
y=model(torch.from_numpy(x).float().resize(50,1))
y=y.detach().numpy()
plt.scatter(response,satisfy,s=1,marker='+',c=(0,0,1))
plt.plot(x,10*y)
plt.show()

NameError: name 'np' is not defined

In [164]:
model.l1.weight

Parameter containing:
tensor([[0.4241],
        [0.6125],
        [0.7225],
        [0.1596],
        [0.1607]], requires_grad=True)

In [165]:
model.l2.weight

Parameter containing:
tensor([[ 0.1602,  0.1650,  0.3963, -0.5008, -0.8633]], requires_grad=True)